# Amazon Data Extracter General

In [1]:
#importing used packages
import csv
from selenium import webdriver 
import pandas as pd
import numpy as np
import time
import re

In [2]:
#path of web driver
PATH = "C:\Program Files (x86)\chromedriver.exe"

## Function to get all products link with addition argument as number of pages want to scrap

In [3]:
#for product links
def get_product_links(driver,number_of_pages,product_name):
    
    
    product_name = re.subn(" ","+",product_name)[0]  #link has no spaces so we are changing sapce with "+"     
    links=[]
    i=1
    
    #extracting product links from different webpages
    while(i<number_of_pages+1):
        time.sleep(0.5)
        
        #opening different webpages by changing product name and page number for link extraction
        url = "https://www.amazon.in/s?k={}&page=".format(product_name.lower())+str(i)+"&qid=1634839581&ref=sr_pg_1"
        driver.get(url) 
        time.sleep(0.5)
        
        #extracting all the link attached to products name
        all_a = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
        for a in all_a:
            links.append(a.get_attribute("href"))
        i+=1
        
    return links


## Function to get all features of the first product

In [4]:
#for feature names
def get_features_from_first_product(driver,link):
    driver.get(link)
    col = ['Name','Price']
    for x in range(1,100):
        try:
            #extracting all the feactures(values in dark column)
            c = driver.find_element_by_xpath('//*[@id="productDetails_techSpec_section_1"]/tbody/tr['+ str(x) +']/th').text
            col.append(c)
        except:
            break
    return col

## Function to get data from amazon

In [19]:
#for getting data
def get_data_from_amazon(page_number):
    
    #get Product name
    product_name = input("Enter the product name: ")
    
    driver = webdriver.Chrome(PATH) #open browser
    
    link = get_product_links(driver,page_number,product_name) #extracting all products link
    print(len(link))
    a = np.arange(len(link))
    
    col = get_features_from_first_product(driver,link[0]) #extracting features
    product = pd.DataFrame(index = a,columns=col)
    print(col)
    
    #extracting information from all links
    for i in a:
            driver.get(link[i])
            time.sleep(0.5)
            
            #extracting product name
            try:
                name = driver.find_element_by_xpath('//*[@id="productTitle"]').text 
                product.loc[i,'Name'] = name
            except:
                None
                
            #extracting product price
            try:
                
                price = driver.find_element_by_xpath('//td[@class="a-span12"]/span[1]').text or driver.find_element_by_xpath('//*[@id="priceblock_dealprice"]').text   
                #for price
                product.loc[i,'Price'] = price
            except:
                None
                
            try:
                #extracting all the features of the current product on the webpage
                c = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//th[@class="a-color-secondary a-size-base prodDetSectionEntry"]')
                
                #extracting all the values corresponding to the features 
                d = driver.find_elements_by_xpath('//div[@class="a-expander-content a-expander-section-content a-section-expander-inner"]//td[@class="a-size-base prodDetAttrValue"]')
                
                #saving the values of the selected featues in the database 
                for x in range(len(c)):
                    #converting to text
                    if c[x].text in col:
                        product.loc[i,c[x].text] = d[x].text
            except:
                print(link[i])

    driver.quit()
    product.to_csv(product_name+'.csv')
    return product

In [20]:
product = get_data_from_amazon(1) #passing page number for easy test

Enter the product name: sony camera
18
['Name', 'Price', 'Brand', 'Manufacturer', 'Model', 'Model Name', 'Model Year', 'Product Dimensions', 'Batteries', 'Item model number', 'Hardware Interface', 'Special Features', 'Mounting Hardware', 'Number Of Items', 'Standing screen display size', 'Display Type', 'Image Stabilization', 'Optical Zoom', 'Digital Zoom', 'Max Resolution', 'Optical Sensor Resolution', 'Maximum Shutter Speed', 'Minimum Shutter Speed', 'Min Focal Length', 'Video Capture Resolution', 'Batteries Included', 'Batteries Required', 'Battery Cell Composition', 'Continuous Shooting Speed', 'Device interface - primary', 'Form Factor', 'Lens Type', 'Viewfinder Type', 'Includes Rechargable Battery', 'Has Self Timer', 'Includes External Memory', 'Manufacturer', 'Country of Origin', 'Imported By', 'Item Weight']


In [21]:
product

,Name,Price,Brand,Manufacturer,Model,Model Name,Model Year,Product Dimensions,Batteries,Item model number,...,Form Factor,Lens Type,Viewfinder Type,Includes Rechargable Battery,Has Self Timer,Includes External Memory,Manufacturer,Country of Origin,Imported By,Item Weight
0,Sony DSC W830 Cyber-Shot 20.1 MP Point and Sho...,"₹9,999.00",Sony,Sony Imaging Products & Solutions Inc.,SNY_W830PS_BLK,Cybershot,2016,5.3 x 2.3 x 9.3 cm; 120 Grams,1 Lithium ion batteries required. (included),SNY_W830PS_BLK,...,DSLR,Zoom,Electronic,Yes,Yes,No,Sony Imaging Products & Solutions Inc.,China,"Sony Imaging Products & Solutions Inc., 1-7-1 ...",120 g
1,Sony Cybershot DSC-WX500/B 18.2MP Digital Came...,"₹20,999.00",Sony,Sony,DSC-WX500,Cyber-shot W Series,2015,3.6 x 10.2 x 5.8 cm; 209 Grams,1 Lithium ion batteries required. (included),DSC-WX500,...,Compact,Zoom,Electronic,Yes,Yes,NaN,Sony,China,"Sony Imaging Products & Solutions Inc., 1-7-1 ...",209 g
2,Sony Alpha ILCE 6000L 24.3 MP Mirrorless Digit...,"₹43,190.00",Sony,Sony,ILCE-6000L,Alpha,2014,4.5 x 12 x 6.7 cm; 344 Grams,1 Lithium ion batteries required. (included),ILCE-6000L,...,Mirrorless,"Standard, Wide Angle",Electronic,NaN,Yes,NaN,Sony,Thailand,"Sony Imaging Products & Solutions Inc., 1-7-1 ...",344 g
3,Sony DSC-H300/BC E32 Cyber-Shot Point & Shoot ...,NaN,Sony,Sony Imaging Products & Solutions Inc.,DSC-H300B,Cybershot,2013,9.17 x 12.75 x 8.9 cm; 590 Grams,1 Lithium Metal batteries required. (included),DSC-H300B,...,DSLR,Zoom,Electronic,NaN,Yes,NaN,Sony Imaging Products & Solutions Inc.,NaN,"Sony Imaging Products & Solutions Inc., 1-7-1 ...",590 g
4,(Renewed) Sony SNY_W830PS_BLK Cyber-Shot 20.1 ...,"₹8,500.00",Sony,Sony,SNY_W830PS_BLK,Standard,NaN,5.3 x 2.3 x 9.3 cm; 118 Grams,1 Lithium ion batteries required. (included),SNY_W830PS_BLK,...,NaN,Zoom,NaN,NaN,NaN,NaN,Sony,NaN,NaN,118 g


In [22]:
#checking null values
product.isnull().sum()

Name                            0
Price                           1
Brand                           0
Manufacturer                    0
Model                           0
Model Name                      0
Model Year                      1
Product Dimensions              0
Batteries                       0
Item model number               0
Hardware Interface              4
Special Features                1
Mounting Hardware               0
Number Of Items                 0
Standing screen display size    0
Display Type                    0
Image Stabilization             1
Optical Zoom                    0
Digital Zoom                    0
Max Resolution                  1
Optical Sensor Resolution       0
Maximum Shutter Speed           2
Minimum Shutter Speed           1
Min Focal Length                0
Video Capture Resolution        0
Batteries Included              0
Batteries Required              0
Battery Cell Composition        0
Continuous Shooting Speed       2
Device interfa